<a href="https://colab.research.google.com/github/RohitSh26/neural_networks/blob/master/MakeMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

import
 matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2022-10-31 02:50:38--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt.2’

names.txt.2         100%[===================>] 222.80K  --.-KB/s    in 0.02s   

2022-10-31 02:50:38 (11.2 MB/s) - ‘names.txt.2’ saved [228145/228145]



In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
len(words)

32033

In [ ]:
chars = sorted(list(set(''.join(words))))

In [ ]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 27

itos = {i:s for s,i in stoi.items()}

In [ ]:
# build the data set

BLOCK_SIZE = 3

def build_dataset(words):
  X, Y = [], []

  for w in words:

    # create a context
    context = [0] * BLOCK_SIZE

    for ch in w + '.':
      # get index of the character
      ix = stoi[ch]

      X.append(context)
      Y.append(ix)

      context = context[1:] + [ix]


  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

In [ ]:
import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xde, Yde = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
Xtr.shape, Xtr.dtype, Ytr.shape, Ytr.dtype

(torch.Size([182625, 3]), torch.int64, torch.Size([182625]), torch.int64)

In [ ]:
# generator
g = torch.Generator().manual_seed(2147483647)
# embeddings
C = torch.randn((27, 10), generator=g)

W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn((27), generator=g)

parameters = [C, W1, b1, W2, b2]

In [ ]:
# number of parameters
sum(p.nelement() for p in parameters)

11897

In [ ]:
for p in parameters:
  p.requires_grad = True

In [ ]:
# learning rate
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
# training loop
